---

# Caso práctico: Predicción de Picos de Intensidad de impactos en un Molino SAG

---

# 1. Importación de Librerías

In [ ]:
import matplotlib #.pyplot as plt1
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.tsa.api as smt
import warnings
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error 

In [ ]:
warnings.filterwarnings("ignore")
%matplotlib inline

# 2. Importación de Datos

In [ ]:
df = pd.read_csv('data/picos_intensidad_clase.csv',delimiter=';')
df.head()

# 3. Limpieza de Datos

In [ ]:
##Tratamiento de datos invalidos
#-------------------------------------------
titles=list(df)
for col in titles:
    #Eliminar filas cuyo valor en columna tenga un dato vacio
    df=df.dropna(subset=[col])
    #Eliminar valores invalidos
    df = df.drop(df[(df[col]=='Bad') | (df[col]=='Shutdown') | (df[col]=='I/O Timeout')| (df[col]=='[-11059] No Good Data For Calculation') | (df[col]==' ')].index)
#Filtro Sag encendido
df = df.drop(df[df['SAG Run Sts']=='Stop'].index)
df = df.drop(['SAG Run Sts'], axis=1)
df.isnull().sum(axis=0)

df = df.drop(['Fecha'], axis=1) # Igual a: df.drop(['Fecha'], axis='columns', inplace=True)
df=df.astype(float).round(2)

In [ ]:
#1. Analisis de Correlaciones de Pearson
corr_mat=df.corr(method='pearson')
plt.figure(figsize=(10,8))
sns.heatmap(corr_mat,vmax=1,square=True,annot=True,center= 0,cmap='coolwarm') # other params: linewidths=3, linecolor='black'

# 4. Modelamiento Predictivo

In [ ]:
# Target: Variable que intentamos pronosticar
target_col = "Intensidad de impactos"

# Definicion de variables de estudio
X = df.loc[:, df.columns != target_col]
y = df.loc[:, target_col]

#Realizamos una selección de muestra ENTRENAMIENTO Y VALIDACION aletoria y comparables muestras, con 
#fines de validar el aprendizaje de nuestro modelo
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.30, 
                                                    random_state=42)

In [ ]:
def indicadores_regresion(y_test,y_pred):
    # Calculamos el Root Mean Square Error: Suma(Y_reales-Y_predichos)^2/n
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)

    # Definimos y calculamos el MAPE (mean_absolute_percentage_error)
    y_test, y_pred = np.array(y_test), np.array(y_pred)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

    return rmse,mape

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures


## 4.1. Modelo de Regresion Lineal

In [ ]:
# Entrenamiento del modelo
model_regression_lineal = LinearRegression()
model_regression_lineal.fit(X_train, y_train)

# Predecir el conjunto de test
y_pred = model_regression_lineal.predict(X_test)
residual = y_test - y_pred

# Mostrar indicadores de precision
rmse, mape = indicadores_regresion(y_test,y_pred)
print(f'RMSE: {rmse} - MAPE: {mape}')

# Gráfico: Validación del Residual como una Distribución Normal
ax = sns.distplot(residual)

# Gráfico: Validación de Homocedasticidad
fig, ax = plt.subplots(figsize=(20,10))
ax.scatter(y_pred, residual)

In [ ]:
# regression coefficients and R-squared value of model
print(model_regression_lineal.intercept_, model_regression_lineal.coef_, model_regression_lineal.score(X_train, y_train))

In [ ]:
ecuacion_regresion = dict(zip(X_train.columns, model_regression_lineal.coef_))
ecuacion_regresion['INTERCEPTO'] = model_regression_lineal.intercept_
ecuacion_regresion

## 4.2 Modelo de Regresión Polinómica

In [ ]:
# Transformación polinómica de datos
poly_reg = PolynomialFeatures(degree = 2)
X_train_poly = poly_reg.fit_transform(X_train)
X_test_poly = poly_reg.fit_transform(X_test)

# Entrenamiento de modelo
model_regression_lineal_transform = LinearRegression()
model_regression_lineal_transform.fit(X_train_poly, y_train)

# Predecir el conjunto de test
y_pred = model_regression_lineal_transform.predict(X_test_poly)
residual = y_test - y_pred

# Mostrar indicadores de precision
rmse, mape = indicadores_regresion(y_test,y_pred)
print(f'RMSE: {rmse} - MAPE: {mape}')

# Gráfico: Validación del Residual como una Distribución Normal
ax = sns.distplot(residual)

# Gráfico: Validación de Homocedasticidad
fig, ax = plt.subplots(figsize=(20,10))
ax.scatter(y_pred, residual)

In [ ]:
# Transformación polinómica de datos
poly_reg = PolynomialFeatures(degree = 3)
X_train_poly = poly_reg.fit_transform(X_train)
X_test_poly = poly_reg.fit_transform(X_test)

# Entrenamiento de modelo
model_regression_lineal_transform = LinearRegression()
model_regression_lineal_transform.fit(X_train_poly, y_train)

# Predecir el conjunto de test
y_pred = model_regression_lineal_transform.predict(X_test_poly)
residual = y_test - y_pred

# Mostrar indicadores de precision
rmse, mape = indicadores_regresion(y_test,y_pred)
print(f'RMSE: {rmse} - MAPE: {mape}')

# Gráfico: Validación del Residual como una Distribución Normal
ax = sns.distplot(residual)

# Gráfico: Validación de Homocedasticidad
fig, ax = plt.subplots(figsize=(20,10))
ax.scatter(y_pred, residual)

In [ ]:
# Transformación polinómica de datos
poly_reg = PolynomialFeatures(degree = 4)
X_train_poly = poly_reg.fit_transform(X_train)
X_test_poly = poly_reg.fit_transform(X_test)

# Entrenamiento de modelo
model_regression_lineal_transform = LinearRegression()
model_regression_lineal_transform.fit(X_train_poly, y_train)

# Predecir el conjunto de test
y_pred = model_regression_lineal_transform.predict(X_test_poly)
residual = y_test - y_pred

# Mostrar indicadores de precision
rmse, mape = indicadores_regresion(y_test,y_pred)
print(f'RMSE: {rmse} - MAPE: {mape}')

# Gráfico: Validación del Residual como una Distribución Normal
ax = sns.distplot(residual)

# Gráfico: Validación de Homocedasticidad
fig, ax = plt.subplots(figsize=(20,10))
ax.scatter(y_pred, residual)

In [ ]:
X_train_poly.shape

In [ ]:
# R-squared value of model
model_regression_lineal_transform.score(X_train_poly, y_train)